In [24]:
#Imports
import sqlite3
from sqlite3 import Error

import pandas as pd
from datetime import datetime

## Create Database and Tables

In [25]:

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn
 

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def main():
    database = r"active_learning_20191210.db"
 
    sql_create_training_log = """ CREATE TABLE IF NOT EXISTS training_log (
                                        training_id integer PRIMARY KEY,
                                        training_time datetime,
                                        file_path string,
                                        from_scratch boolean
                                    ); """
 
    sql_create_image_log = """ CREATE TABLE IF NOT EXISTS image_log (
                                        image_id integer PRIMARY KEY,
                                        file_path string,
                                        time_uploaded datetime
                                    ); """
 
    sql_create_map_log = """ CREATE TABLE IF NOT EXISTS map_log (
                                        map_id integer PRIMARY KEY,
                                        file_path string,
                                        time_created datetime,
                                        is_manual boolean
                                    ); """
 
    sql_create_train_to_image_log = """CREATE TABLE IF NOT EXISTS train_to_image_log (
                                    entry_id integer PRIMARY KEY,
                                    image_id integer,
                                    training_id integer,
                                    FOREIGN KEY (image_id) REFERENCES image_log (image_id),
                                    FOREIGN KEY (training_id) REFERENCES training_log (training_id)
                                );"""
    
    sql_create_image_to_map_log = """CREATE TABLE IF NOT EXISTS image_to_map_log (
                                    entry_id integer PRIMARY KEY,
                                    image_id integer,
                                    map_id integer,
                                    FOREIGN KEY (image_id) REFERENCES image_log (image_id),
                                    FOREIGN KEY (map_id) REFERENCES map_log (map_id)
                                );"""
 
    # create a database connection
    conn = create_connection(database)
 
    # create tables
    if conn is not None:
        # create training_log
        create_table(conn, sql_create_training_log)
 
        # create image_log
        create_table(conn, sql_create_image_log)
        
        # create map_log
        create_table(conn, sql_create_map_log)
        
        # create train_to_image_log
        create_table(conn, sql_create_train_to_image_log)
        
        # create image_to_map_log
        create_table(conn, sql_create_image_to_map_log)
    else:
        print("Error! cannot create the database connection.")
 


In [26]:
main()

In [27]:
#Add entries in image log

con = sqlite3.connect("active_learning_20191210.db")

cur = con.cursor()

now = datetime.now()

cur_date = str(now.year) + '_' + str(now.month) + '_' + str(
            now.day) + '_' + str(now.hour) + '_' + str(now.minute)

for slice_id in range(100, 106):
    
    file_path = "../prototype/assets/Site6_031923___" + str(slice_id)
    image_log_entry = (file_path, cur_date)
    cur.execute('INSERT INTO image_log (file_path, time_uploaded) VALUES' + str(image_log_entry))

con.commit()

con.close()

In [53]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("active_learning_20191210.db")
df = pd.read_sql_query("SELECT * from image_log", con)

# Verify that result of SQL query is stored in the dataframe
print(df)

con.close()

   image_id                               file_path   time_uploaded
0         1  ../prototype/assets/Site6_031923___100  2020_1_24_14_7
1         2  ../prototype/assets/Site6_031923___101  2020_1_24_14_7
2         3  ../prototype/assets/Site6_031923___102  2020_1_24_14_7
3         4  ../prototype/assets/Site6_031923___103  2020_1_24_14_7
4         5  ../prototype/assets/Site6_031923___104  2020_1_24_14_7
5         6  ../prototype/assets/Site6_031923___105  2020_1_24_14_7


In [54]:
con = sqlite3.connect("active_learning_20191210.db")
df = pd.read_sql_query("SELECT * from map_log", con)

# Verify that result of SQL query is stored in the dataframe
print(df)

con.close()

     map_id                                file_path     time_created  \
0         1  data/train/label/2020_1_24_14_11_1.jpeg  2020_1_24_14_11   
1         2  data/train/label/2020_1_24_14_11_2.jpeg  2020_1_24_14_11   
2         3  data/train/label/2020_1_24_14_11_3.jpeg  2020_1_24_14_11   
3         4  data/train/label/2020_1_24_14_11_4.jpeg  2020_1_24_14_11   
4         5  data/train/label/2020_1_24_14_11_5.jpeg  2020_1_24_14_11   
..      ...                                      ...              ...   
241     242  output/2020_1_30_14_22/2_prediction.npy  2020_1_30_14_22   
242     243  output/2020_1_30_14_22/3_prediction.npy  2020_1_30_14_22   
243     244  output/2020_1_30_14_22/4_prediction.npy  2020_1_30_14_22   
244     245  output/2020_1_30_14_22/5_prediction.npy  2020_1_30_14_22   
245     246  output/2020_1_30_14_22/6_prediction.npy  2020_1_30_14_22   

     is_manual  
0            1  
1            1  
2            1  
3            1  
4            1  
..         ...  
241 

In [55]:
con = sqlite3.connect("active_learning_20191210.db")
df = pd.read_sql_query("SELECT * from image_to_map_log", con)

# Verify that result of SQL query is stored in the dataframe
print(df)

con.close()

     entry_id  image_id  map_id
0           1         1       1
1           2         2       2
2           3         3       3
3           4         4       4
4           5         5       5
..        ...       ...     ...
241       242         2     242
242       243         3     243
243       244         4     244
244       245         5     245
245       246         6     246

[246 rows x 3 columns]


In [56]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("active_learning_20191210.db")
df = pd.read_sql_query("SELECT * from training_log", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

   training_id                  training_time  \
0            1  Fri, 24 Jan 2020 14:12:19 GMT   
1            2  Mon, 27 Jan 2020 14:42:26 GMT   
2            3  Mon, 27 Jan 2020 14:55:51 GMT   
3            4  Mon, 27 Jan 2020 15:13:02 GMT   
4            5  Mon, 27 Jan 2020 15:23:57 GMT   

                                      file_path  from_scratch  
0  models/unet_stroke_2020_1_24_14_12_init.hdf5             1  
1  models/unet_stroke_2020_1_27_14_42_init.hdf5             1  
2  models/unet_stroke_2020_1_27_14_55_init.hdf5             1  
3  models/unet_stroke_2020_1_27_15_13_init.hdf5             1  
4  models/unet_stroke_2020_1_27_15_23_init.hdf5             1  


In [57]:
con = sqlite3.connect("active_learning_20191210.db")

df = pd.read_sql_query("SELECT * from train_to_image_log", con)

# Verify that result of SQL query is stored in the dataframe
print(df)

con.close()

     entry_id  image_id  training_id
0           1         1            1
1           2         2            1
2           3         3            1
3           4         4            1
4           5         5            1
..        ...       ...          ...
103       104         2           18
104       105         3           18
105       106         4           18
106       107         5           18
107       108         6           18

[108 rows x 3 columns]


In [22]:
#Look at all tables
con = sqlite3.connect('active_learning_20191210.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('training_log',), ('image_log',), ('map_log',), ('train_to_image_log',), ('image_to_map_log',)]


## Input into tables

In [77]:
#Function to prep the columns for the SQL query
def prep_cols(cols):
    
    #init string
    final_str = ''
    
    #loop through columns and construct string
    for i in cols:
        final_str = final_str + i + ','
        
    #return
    return final_str[:-1]

In [86]:
def create_entry(conn, table_name, task):
    """
    Create a new entry
    :param conn:
    :param task:
    :return:
    """
    
    #Read in table
    df = pd.read_sql_query("SELECT * from " + db_name + " ", conn)

    #get list of columns
    columns = list(df.columns)

    #Construct sql query
    sql = ' INSERT INTO ' + db_name + '(' + prep_cols(columns) + ')'  + ' VALUES(?, ?, ?, ?) '

#     sql = ''' INSERT INTO training_log(training_id, training_time, file_path, from_scratch) VALUES(?, ?, ?, ?) '''

    #execute
    cur = conn.cursor()
    cur.execute(sql, task)
    return cur.lastrowid


def main():
    
    #Database name
    database = "active_learning_20191210.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn:
        
        # Specific table and inputs
        table_name = 'training_log'
        task_1 = ('10002', '12/10/2019', 'example/file/path', True)
 
        # create entry
        create_entry(conn, table_name, task_1)
        
    conn.close()


In [87]:
main()

In [88]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("active_learning_20191210.db")
df = pd.read_sql_query("SELECT * from training_log", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

   training_id training_time          file_path  from_scratch
0          100    12/10/2019  example/filt/path             1
1        10000    12/10/2019  example/file/path             1
2        10001    12/10/2019  example/file/path             1
3        10002    12/10/2019  example/file/path             1


## Delete Row

In [41]:
def deleteRecord():
    try:
        con = sqlite3.connect('active_learning_20191210.db')
        cursor = con.cursor()
        print("Connected to SQLite")

        # Deleting single record now
        sql_delete_query = """DELETE from training_log where training_id = 0"""
        cursor.execute(sql_delete_query)
        con.commit()
        print("Record deleted successfully ")
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to delete record from sqlite table", error)
    finally:
        if (con):
            con.close()
            print("the sqlite connection is closed")

deleteRecord()

Connected to SQLite
Record deleted successfully 
the sqlite connection is closed


In [42]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("active_learning_20191210.db")
df = pd.read_sql_query("SELECT * from training_log", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

   training_id training_time          file_path  from_scratch
0          100    12/10/2019  example/filt/path             1
1        10000    12/10/2019  example/file/path             1
